In [27]:
'''Felix Andersson, Janine de Vries, DV2626'''

import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import time


In [28]:
spam_data = pd.read_csv('spambase.data', delimiter=',')
spam_data.head()
spam_data.drop_duplicates(inplace=True)

x = spam_data.iloc[:, :-1]
y = spam_data.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


print("X:\n", x.head()) 
print("Y:\n", y.head())  

X:
       0  0.64  0.64.1  0.1  0.32   0.2   0.3   0.4   0.5   0.6  ...  0.40  \
0  0.21  0.28    0.50  0.0  0.14  0.28  0.21  0.07  0.00  0.94  ...   0.0   
1  0.06  0.00    0.71  0.0  1.23  0.19  0.19  0.12  0.64  0.25  ...   0.0   
2  0.00  0.00    0.00  0.0  0.63  0.00  0.31  0.63  0.31  0.63  ...   0.0   
3  0.00  0.00    0.00  0.0  0.63  0.00  0.31  0.63  0.31  0.63  ...   0.0   
4  0.00  0.00    0.00  0.0  1.85  0.00  0.00  1.85  0.00  0.00  ...   0.0   

   0.41   0.42  0.43  0.778   0.44   0.45  3.756   61   278  
0  0.00  0.132   0.0  0.372  0.180  0.048  5.114  101  1028  
1  0.01  0.143   0.0  0.276  0.184  0.010  9.821  485  2259  
2  0.00  0.137   0.0  0.137  0.000  0.000  3.537   40   191  
3  0.00  0.135   0.0  0.135  0.000  0.000  3.537   40   191  
4  0.00  0.223   0.0  0.000  0.000  0.000  3.000   15    54  

[5 rows x 57 columns]
Y:
 0    1
1    1
2    1
3    1
4    1
Name: 1, dtype: int64


In [29]:
classifiers = {
    "SVM": SVC(),
    "Random Forest": RandomForestClassifier(),
    "KNN": KNeighborsClassifier()
}

time_table = {name: [] for name in classifiers}
avrage_table = {name: [] for name in classifiers}

skf = StratifiedKFold(n_splits=10)

for fold, (train_index, test_index) in enumerate(skf.split(x, y), 1):
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    for name, algorithm in classifiers.items():
        start_time = time.time()
        algorithm.fit(X_train, y_train)
        time_training = time.time() - start_time
        
        y_pred = algorithm.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        avrage_table[name].append(time_training)
        avrage_table[name].append(accuracy)

output = []

for fold_num in range(10):
    fold_data = []
    for name, scores in avrage_table.items():
        fold_data.append(f"{scores[fold_num]:.4f}")
    output.append(fold_data)

avg_and_std = []
for name, scores in avrage_table.items():
    avg_accuracy = np.mean(scores)
    std_accuracy = np.std(scores)
    avg_and_std.append([f"{avg_accuracy:.4f}", f"{std_accuracy:.4f}"])

output.append([f"avg {item[0]}" for item in avg_and_std])
output.append([f"stdev {item[1]}" for item in avg_and_std])

fold_df = pd.DataFrame(output, columns=list(classifiers.keys()))
fold_df.index = [f"Fold {i}" for i in range(1, 11)] + ['avg', 'stdev']
fold_df.index.name = 'Avrage'

print(fold_df)

                  SVM Random Forest           KNN
Avrage                                           
Fold 1         0.3565        0.4874        0.0022
Fold 2         0.6580        0.9454        0.7506
Fold 3         0.3742        0.4999        0.0021
Fold 4         0.6627        0.9549        0.7648
Fold 5         0.3723        0.5167        0.0022
Fold 6         0.7197        0.9240        0.7625
Fold 7         0.3835        0.5023        0.0021
Fold 8         0.7648        0.9501        0.8076
Fold 9         0.3729        0.4930        0.0021
Fold 10        0.6793        0.9477        0.8005
avg        avg 0.5437    avg 0.7162    avg 0.3903
stdev    stdev 0.1676  stdev 0.2199  stdev 0.3889


In [30]:
''' 
Friedman test

'''

classifiers = {
    "SVM": SVC(),
    "Random Forest": RandomForestClassifier(),
    "KNN": KNeighborsClassifier()
}

# Create dictionaries to store time, accuracy, and F1 scores
metrics_table = {name: {'time': [], 'accuracy': [], 'f1': []} for name in classifiers}

# Stratified K-Fold cross-validation
skf = StratifiedKFold(n_splits=10)

# Loop through each fold and each classifier
for fold, (train_index, test_index) in enumerate(skf.split(x, y), 1):
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Loop through classifiers
    for name, algorithm in classifiers.items():
        # Record start time for training
        start_time = time.time()
        algorithm.fit(X_train, y_train)
        time_training = time.time() - start_time  # Time taken for training

        # Predictions and metrics
        y_pred = algorithm.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')

        # Store training time, accuracy, and F1 score
        metrics_table[name]['time'].append(time_training)
        metrics_table[name]['accuracy'].append(accuracy)
        metrics_table[name]['f1'].append(f1)

# Prepare output for accuracy, F1, and training time


# Add the averages and standard deviations to the output tables


# Convert accuracy, F1, and time tables to DataFrame


In [ ]:
output_accuracy = []
output_f1 = []
output_time = []

# Loop through each fold and collect data for accuracy, F1, and time
for fold_num in range(10):
    fold_data_accuracy = []
    fold_data_f1 = []
    fold_data_time = []
    
    for name in classifiers.keys():
        # Append accuracy, F1 score, and time values for each classifier
        fold_data_accuracy.append(f"{metrics_table[name]['accuracy'][fold_num]:.4f}")
        fold_data_f1.append(f"{metrics_table[name]['f1'][fold_num]:.4f}")
        fold_data_time.append(f"{metrics_table[name]['time'][fold_num]:.4f}")
    
    output_accuracy.append(fold_data_accuracy)
    output_f1.append(fold_data_f1)
    output_time.append(fold_data_time)

# Prepare averages and standard deviations for accuracy, F1, and time
avg_and_std_accuracy = []
avg_and_std_f1 = []
avg_and_std_time = []

for name in classifiers.keys():
    avg_accuracy = np.mean(metrics_table[name]['accuracy'])
    std_accuracy = np.std(metrics_table[name]['accuracy'])
    avg_and_std_accuracy.append([f"{avg_accuracy:.4f}", f"{std_accuracy:.4f}"])
    
    avg_f1 = np.mean(metrics_table[name]['f1'])
    std_f1 = np.std(metrics_table[name]['f1'])
    avg_and_std_f1.append([f"{avg_f1:.4f}", f"{std_f1:.4f}"])
    
    avg_time = np.mean(metrics_table[name]['time'])
    std_time = np.std(metrics_table[name]['time'])
    avg_and_std_time.append([f"{avg_time:.4f}", f"{std_time:.4f}"])

    
output_accuracy.append([f"avg {item[0]}" for item in avg_and_std_accuracy])
output_accuracy.append([f"stdev {item[1]}" for item in avg_and_std_accuracy])

output_f1.append([f"avg {item[0]}" for item in avg_and_std_f1])
output_f1.append([f"stdev {item[1]}" for item in avg_and_std_f1])

output_time.append([f"avg {item[0]}" for item in avg_and_std_time])
output_time.append([f"stdev {item[1]}" for item in avg_and_std_time])

accuracy_df = pd.DataFrame(output_accuracy, columns=list(classifiers.keys()))
f1_df = pd.DataFrame(output_f1, columns=list(classifiers.keys()))
time_df = pd.DataFrame(output_time, columns=list(classifiers.keys()))

# Add fold numbers to the DataFrame index
accuracy_df.index = [f"Fold {i}" for i in range(1, 11)] + ['avg', 'stdev']
accuracy_df.index.name = 'Avrage'

f1_df.index = [f"Fold {i}" for i in range(1, 11)] + ['avg', 'stdev']
f1_df.index.name = 'Avrage'

time_df.index = [f"Fold {i}" for i in range(1, 11)] + ['avg', 'stdev']
time_df.index.name = 'Avrage'

In [ ]:


# Display the results
print("Accuracy and F1 Scores Table:")
print(accuracy_df)



Accuracy and F1 Scores Table:
                  SVM Random Forest           KNN
Avrage                                           
Fold 1         0.6580        0.9501        0.7506
Fold 2         0.6627        0.9525        0.7648
Fold 3         0.7197        0.9311        0.7625
Fold 4         0.7648        0.9549        0.8076
Fold 5         0.6793        0.9501        0.8005
Fold 6         0.6936        0.9525        0.8029
Fold 7         0.7530        0.9572        0.7981
Fold 8         0.7316        0.9715        0.8242
Fold 9         0.7577        0.8884        0.7126
Fold 10        0.6667        0.8548        0.7619
avg        avg 0.7087    avg 0.9363    avg 0.7786
stdev    stdev 0.0396  stdev 0.0345  stdev 0.0320


In [34]:
print("\nF1 Score Table:")
print(f1_df)




F1 Score Table:
                  SVM Random Forest           KNN
Avrage                                           
Fold 1         0.6337        0.9499        0.7496
Fold 2         0.6388        0.9524        0.7645
Fold 3         0.6907        0.9304        0.7602
Fold 4         0.7514        0.9547        0.8053
Fold 5         0.6540        0.9500        0.8009
Fold 6         0.6761        0.9526        0.8031
Fold 7         0.7330        0.9571        0.7951
Fold 8         0.7085        0.9714        0.8225
Fold 9         0.7292        0.8893        0.7155
Fold 10        0.6344        0.8544        0.7619
avg        avg 0.6850    avg 0.9362    avg 0.7779
stdev    stdev 0.0420  stdev 0.0344  stdev 0.0310


In [35]:
print("\nTraining Time Table:")
print(time_df)


Training Time Table:
                  SVM Random Forest           KNN
Avrage                                           
Fold 1         0.3679        0.4952        0.0021
Fold 2         0.3810        0.4964        0.0021
Fold 3         0.3603        0.5001        0.0024
Fold 4         0.3868        0.4919        0.0021
Fold 5         0.3703        0.4907        0.0021
Fold 6         0.3779        0.5071        0.0025
Fold 7         0.3911        0.5015        0.0022
Fold 8         0.4047        0.5089        0.0024
Fold 9         0.3870        0.5076        0.0021
Fold 10        0.3710        0.4979        0.0022
avg        avg 0.3798    avg 0.4997    avg 0.0022
stdev    stdev 0.0124  stdev 0.0062  stdev 0.0001
